In [ ]:
# setup paths
import sys
basedir = '/Users/nickm/thesis/icml2023paper/1d_euler'
readwritedir = '/Users/nickm/thesis/icml2023paper/1d_euler'

sys.path.append('{}/core'.format(basedir))
sys.path.append('{}/simulate'.format(basedir))
sys.path.append('{}/ml'.format(basedir))

In [ ]:
# import external packages
import jax
import jax.numpy as jnp
import numpy as onp
from jax import config, vmap
config.update("jax_enable_x64", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from initialconditions import get_a0, f_init_lax_shock_tube, shock_tube_problem_1, density_propagation
from simparams import CoreParams, SimulationParams
from simulations import ShockTubeSim
from helper import get_rho, get_u, get_p
from trajectory import get_trajectory_fn, get_inner_fn

In [ ]:
def get_core_params(Lx = 1.0, gamma = 5/3, bc = 'periodic', fluxstr = 'laxfriedrichs'):
    return CoreParams(Lx, gamma, bc, fluxstr)


def get_sim_params(name = "test", cfl_safety=0.3, rk='ssp_rk3'):
    return SimulationParams(name, basedir, readwritedir, cfl_safety, rk)


def plot_a(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 3, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_title(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_title(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])

### Shock Tube Problem 1

JE2: http://ammar-hakim.org/sj/je/je2/je2-euler-shock.html

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'roe'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.9, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_1(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.2
outer_steps = 2

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 1.5, 1.1]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)

### Density Propagation

Initialize the density to be a higher value, with constant speed and pressure, and see what happens.

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'periodic', 'fluxstr': 'laxfriedrichs'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.95, 'rk' : 'forward_euler'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = density_propagation(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.1
outer_steps = 4

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
plot_a(a0, core_params)
plot_a(trajectory[-1], core_params)